In [1]:
import numpy as np
import scipy.signal as sps
import sympy as sym 
import scipy.optimize as sopt


In [2]:
eta_d,eta_q = sym.symbols('eta_d,eta_q')
i_d,i_q,v_dc = sym.symbols('i_d,i_q,v_dc')
v_sd,v_sq = sym.symbols('v_sd,v_sq')
L,R,C,omega = sym.symbols('L,R,C,omega')


In [41]:
di_d  = 1/L*(eta_d*v_dc/2 - R*i_d - L*omega*i_q - v_sd)
di_q  = 1/L*(eta_q*v_dc/2 - R*i_q + L*omega*i_d - v_sq)
dv_dc = 3/4*C*(eta_d*i_d - eta_q*i_q)

f = sym.Matrix([[di_d],[di_q],[dv_dc]])
x = sym.Matrix([[i_d],[i_q],[v_dc]])

F_x = f.jacobian(x)

F_x

Matrix([
[        -R/L,        -omega, eta_d/(2*L)],
[       omega,          -R/L, eta_q/(2*L)],
[0.75*C*eta_d, -0.75*C*eta_q,           0]])

In [4]:

from sympy.utilities.lambdify import lambdify


func = lambdify(x, f,'numpy') # returns a numpy-ready function

print(func(1,2,3))

[[(-2*L*omega - R + 3*eta_d/2 - v_sd)/L]
 [(L*omega - 2*R + 3*eta_q/2 - v_sq)/L]
 [0.75*C*(eta_d - 2*eta_q)]]


## $\sf i_d$-$\sf i_q$ mode

In [5]:
params = {'L':2.5e-3,
          'R':1.0,
          'C':2000.0e-6,
          'omega':2.0*np.pi*50,
          'v_sq':np.sqrt(2./3)*400.0,
          'v_sd':0.0,
          'v_dc':700.0
         }

# sym.pretty_print(f.subs(params))

u = sym.Matrix([[i_d],[i_q],[eta_d],[eta_q]])

f_ss = lambdify(u, f[0:2,0].subs(params),'numpy') # returns a numpy-ready function

def eqs_ss(x):
    
    i_d = 0.0
    i_q = 20.0
    
    eta_d, eta_q = x[0],x[1]
    
    return f_ss(i_d, i_q,eta_d,eta_q )

x = sopt.broyden1(eqs_ss, [0,1], f_tol=1e-8)

print('eta_d = {:2.3f}, eta_q = {:2.3f}'.format(x[0],x[1]))

eta_d = 0.045, eta_q = 0.990


In [6]:
x[0]

0.044879895051282592

## $\sf p$-$\sf q$ mode

In [7]:
params = {'L':2.5e-3,
          'R':1.0,
          'C':2000.0e-6,
          'omega':2.0*np.pi*50,
          'v_sq':np.sqrt(2./3)*400.0,
          'v_sd':0.0,
          'v_dc':700.0
         }

p_ref,q_ref = sym.symbols('p_ref,q_ref')

eq_p = 3/2*(v_sd*i_d + v_sq*i_q) - p_ref
eq_q = 3/2*(v_sd*i_q - v_sq*i_d) - q_ref

f_pq =  sym.Matrix([[eq_p],[eq_q]]) 
# sym.pretty_print(f.subs(params))

u = sym.Matrix([[p_ref],[q_ref],[i_d],[i_q],[eta_d],[eta_q]])

f_ss = lambdify(u, f[0:2,0].col_join(f_pq).subs(params),'numpy') # returns a numpy-ready function

def eqs_ss(x):
    
    p_ref = 15.0e3
    q_ref = 0.0e3
    
    i_d, i_q = x[0],x[1]
    eta_d, eta_q = x[2],x[3]
    
    return f_ss(p_ref,q_ref, i_d, i_q,eta_d,eta_q )

x = sopt.broyden1(eqs_ss, [0,1,2,3], f_tol=1e-8)

print('eta_d = {:2.3f}, eta_q = {:2.3f}'.format(x[2],x[3]))

eta_d = 0.069, eta_q = 1.021


## $\sf i_d$-$\sf v_{dc}$ mode

In [36]:
params = {'L':2.5e-3,
          'R':1.0,
          'C':2000.0e-6,
          'omega':2.0*np.pi*50,
          'v_sq':np.sqrt(2./3)*400.0,
          'v_sd':0.0,
          'i_d':-20.0,
          'v_dc':700.0
          }

# sym.pretty_print(f.subs(params))

u = sym.Matrix([[i_q],[eta_d],[eta_q]])

f_ss = lambdify(u, f.subs(params),'numpy') # returns a numpy-ready function


def eqs_ss(x):
    
   
    eta_d, eta_q, i_q = x[0],x[1],x[2]
   
    return f_ss(i_q, eta_d,eta_q )


    
x = sopt.broyden1(eqs_ss, [0,1,10],f_tol=1e-8)



print('eta_d = {:2.3f}, eta_q = {:2.3f}, i_q = {:2.3f}'.format(x[0],x[1],x[2]))

params.update({'eta_d':x[0],'eta_q':x[1],'i_q':x[2]})

p = 3/2*(v_sd*i_d + v_sq*i_q) 
q = 3/2*(v_sd*i_q - v_sq*i_d) 


f_p= lambdify([], p.subs(params),'numpy') # returns a numpy-ready function
f_q= lambdify([], q.subs(params),'numpy') # returns a numpy-ready function

print('p = {:2.3f} W, q = {:2.3f} var'.format(f_p(),f_q()))

eta_d = -0.055, eta_q = 0.981, i_q = 1.114
p = 545.652 W, q = 9797.959 var


In [59]:
f.jacobian(x)

Matrix([
[        -R/L,        -omega, eta_d/(2*L)],
[       omega,          -R/L, eta_q/(2*L)],
[0.75*C*eta_d, -0.75*C*eta_q,           0]])

In [66]:
F_x = f.jacobian(x)
f_A = lambdify([], F_x.subs(params),'numpy')

In [67]:
A = f_A()

In [68]:
np.linalg.eig(A)

(array([ -3.99999797e+02+314.15946506j,  -3.99999797e+02-314.15946506j,
         -4.06136734e-04  +0.j        ]),
 array([[ -5.04147066e-07 +7.07106583e-01j,
          -5.04147066e-07 -7.07106583e-01j,
          -2.38055992e-01 +0.00000000e+00j],
        [  7.07106979e-01 +0.00000000e+00j,
           7.07106979e-01 -0.00000000e+00j,
           2.70653311e-01 +0.00000000e+00j],
        [  1.53879523e-06 +1.35346377e-06j,
           1.53879523e-06 -1.35346377e-06j,
           9.32778714e-01 +0.00000000e+00j]]))